# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-5.52,89,100,2.94,GS,1722738395
1,1,shali,43.1481,45.9019,20.63,53,15,1.70,RU,1722738983
2,2,puerto cabezas,14.0351,-83.3888,25.76,87,61,2.87,NI,1722738803
3,3,kirsanov,52.6549,42.7224,18.82,57,100,2.30,RU,1722738734
4,4,garoowe,8.4054,48.4845,23.16,65,99,9.82,SO,1722738987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_height = 700,
    frame_width = 600,
    size = "Humidity",
    scale = 0.75,
    color="City",
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_condition_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_condition_df = ideal_weather_condition_df.dropna()

# Display sample data
ideal_weather_condition_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,waddan,29.1614,16.1390,26.54,35,0,3.66,LY,1722739017
50,50,franklin,39.9667,-83.0166,23.75,93,0,2.06,US,1722738500
60,60,siwa oasis,29.2041,25.5195,26.79,50,0,2.63,EG,1722739053
68,68,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1722738471
97,97,menominee,45.1078,-87.6143,23.25,78,0,0.00,US,1722739095


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, “Hotel Name,” to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
28,waddan,LY,29.1614,16.1390,35,
50,franklin,US,39.9667,-83.0166,93,
60,siwa oasis,EG,29.2041,25.5195,50,
68,remire-montjoly,GF,4.9167,-52.2667,94,
97,menominee,US,45.1078,-87.6143,78,
101,zakynthos,GR,37.7914,20.8953,64,
136,ihsaniye,TR,40.6909,29.8347,36,
158,cururupu,BR,-1.8283,-44.8683,90,
161,ashland,US,40.8167,-82.2665,95,
201,wilson,US,36.1501,-86.2833,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waddan - nearest hotel: No hotel found
franklin - nearest hotel: The Junto
siwa oasis - nearest hotel: Flower of the city hotel
remire-montjoly - nearest hotel: Complexe Belova
menominee - nearest hotel: AmericInn
zakynthos - nearest hotel: Phoenix Hotel
ihsaniye - nearest hotel: No hotel found
cururupu - nearest hotel: No hotel found
ashland - nearest hotel: Quality Inn
wilson - nearest hotel: Comfort Suites
m'sila - nearest hotel: القصب
tamanrasset - nearest hotel: فندق الأمان
jaguaruana - nearest hotel: Pousada Aconchego
bejar - nearest hotel: Hotel Colón
roma - nearest hotel: Wingate by Wyndham Rome
ceyhan - nearest hotel: Ceyhan Öğretmenevi
raffadali - nearest hotel: Agriturismo Reggia Saracena Agrigento
indian hills - nearest hotel: Courtyard
greenville - nearest hotel: Quality Inn
hun - nearest hotel: فندق الهروج
ketchikan - nearest hotel: Cape Fox Lodge
bulgan - nearest hotel: Bulgan Hotel
safi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
28,waddan,LY,29.1614,16.1390,35,No hotel found
50,franklin,US,39.9667,-83.0166,93,The Junto
60,siwa oasis,EG,29.2041,25.5195,50,Flower of the city hotel
68,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
97,menominee,US,45.1078,-87.6143,78,AmericInn
101,zakynthos,GR,37.7914,20.8953,64,Phoenix Hotel
136,ihsaniye,TR,40.6909,29.8347,36,No hotel found
158,cururupu,BR,-1.8283,-44.8683,90,No hotel found
161,ashland,US,40.8167,-82.2665,95,Quality Inn
201,wilson,US,36.1501,-86.2833,83,Comfort Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    size = "Humidity",
    color ="City",
    scale =.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)